# Setup

Hyperledger Fabric elements are based on docker and we need to setup not just one node of Fabric, but each element separately as a docker containers. This includes:

- orderer - is the service that is independent from other nodes, orderers takes the responsibility of ordering transactions,
- peer - node that belongs to a company/entity that has chaincodes,
- cli - this container that contains the tools and connects the orderer and the peer.

You can extend it with a centrail authority container or build a consortium of peers, but in this example we stick to just one peer and one order to simplify it. The central authority isn't needed too, because the certificates are generated using the fabric tools and shared between the containers.

The ``docker-compose.yaml`` that you have used with the setup script contains the configuration of one peer node, one orderer and the cli. We use the version 3 of docker compose configuration. In the next line we setup a new network where the three containers will be used. This configuration is based on the ``basic-network`` example from ``https://github.com/hyperledger/fabric-samples/tree/master/basic-network``.

version: '3'

networks:
  basic:

In the next section we set the three containers called ``orderer.codete.com``, ``peer0.codete.com``, and ``fabric_cli``. In the first container we use the ``hyperledger/fabric-orderer:1.4`` image as the base. There are several environment variables set, where the most important are ``ORDERER_GENERAL_GENESISFILE`` and ``ORDERER_GENERAL_LOCALMSPDIR``. The genesis block of the orderer system channel is created and included in the configuration of the node before the node can be started. It's the first block of the chain in a channel. The local Membership Service Provider directory contains the credentials to authenticate the transactions. It's attached from our ``/crypto-config/ordererOrganizations/codete.com/orderers/orderer.codete.com/`` directory that was generated by the configuration tool that is described below.

services:
  orderer.codete.com:
    container_name: orderer.codete.com
    image: hyperledger/fabric-orderer:1.4
    environment:
      - ORDERER_GENERAL_LOGLEVEL=info
      - ORDERER_GENERAL_LISTENADDRESS=0.0.0.0
      - ORDERER_GENERAL_GENESISMETHOD=file
      - ORDERER_GENERAL_GENESISFILE=/etc/hyperledger/configtx/genesis.block
      - ORDERER_GENERAL_LOCALMSPID=OrdererMSP
      - ORDERER_GENERAL_LOCALMSPDIR=/etc/hyperledger/msp/orderer/msp
    working_dir: /opt/gopath/src/github.com/hyperledger/fabric/orderer
    command: orderer
    ports:
      - 7050:7050
    volumes:
        - ./config/:/etc/hyperledger/configtx
        - ./crypto-config/ordererOrganizations/codete.com/orderers/orderer.codete.com/:/etc/hyperledger/msp/orderer
        - ./crypto-config/peerOrganizations/codete.com/peers/peer0.codete.com/:/etc/hyperledger/msp/peerOrg1
    networks:
      - basic

The peer node configuration looks quite similar to the orderer configuration. There are just a few major differenes like ``CORE_VM_ENDPOINT`` environment to get the docker address for the communication between the containers. The section ``depends_on`` is new and explains what other containers should be run before the current one. The section ``command`` is a also different and much more complex compared to the previous configuration. It's divided into three parts. In the first one we just start the node. The second and third part are about creating a channel between the orderer and the peer node.

  peer0.codete.com:
    container_name: peer0.codete.com
    image: hyperledger/fabric-peer:1.4
    environment:
      - CORE_VM_ENDPOINT=unix:///host/var/run/docker.sock
      - CORE_PEER_ID=peer0.codete.com
      - CORE_LOGGING_PEER=info
      - CORE_CHAINCODE_LOGGING_LEVEL=info
      - CORE_PEER_LOCALMSPID=Org1MSP
      - CORE_PEER_MSPCONFIGPATH=/etc/hyperledger/msp/peer/
      - CORE_PEER_ADDRESS=peer0.codete.com:7051
      - CORE_VM_DOCKER_HOSTCONFIG_NETWORKMODE=network_basic
    working_dir: /opt/gopath/src/github.com/hyperledger/fabric
    ports:
      - 7051:7051
      - 7053:7053
    volumes:
        - /var/run/:/host/var/run/
        - ./crypto-config/peerOrganizations/org1.codete.com/peers/peer0.org1.codete.com/msp:/etc/hyperledger/msp/peer
        - ./crypto-config/peerOrganizations/org1.codete.com/users:/etc/hyperledger/msp/users
        - ./config:/etc/hyperledger/configtx
    depends_on:
      - orderer.codete.com
    networks:
      - basic
    command: bash -c "
        peer node start &&       CORE_PEER_MSPCONFIGPATH=/etc/hyperledger/msp/users/Admin@codete.com/msp peer0.codete.com peer channel create -o orderer.codete.com:7050 -c channelone -f /etc/hyperledger/configtx/channel.tx &&    CORE_PEER_MSPCONFIGPATH=/etc/hyperledger/msp/users/Admin@codete.com/msp peer0.codete.com peer channel join -b channelone.block"




Finally, the third container is the one where we will build our chaincode and execute it on the peer. The ``dlt-fabric`` image is based on ``hyperledger/fabric-tools``, but contains also the Jupyter notebook installed. 

  fabric_cli:
    container_name: fabric_cli
    image: kprzystalski/dlt-fabric:latest
    tty: true
    environment:
      - GOPATH=/opt/gopath
      - CORE_VM_ENDPOINT=unix:///host/var/run/docker.sock
      - FABRIC_LOGGING_SPEC=info
      - CORE_PEER_ID=cli
      - CORE_PEER_ADDRESS=peer0.codete.com:7051
      - CORE_PEER_LOCALMSPID=Org1MSP
      - CORE_PEER_MSPCONFIGPATH=/opt/gopath/src/github.com/hyperledger/fabric/peer/crypto/peerOrganizations/codete.com/users/Admin@codete.com/msp
      - CORE_CHAINCODE_KEEPALIVE=10
    working_dir: /opt/gopath/src/github.com/hyperledger/fabric/peer
    command: /bin/bash
    volumes:
        - /var/run/:/host/var/run/
        - ./../chaincode/:/opt/gopath/src/github.com/
        - ./crypto-config:/opt/gopath/src/github.com/hyperledger/fabric/peer/crypto/
        - ./carmileage-tracking/:/opt/gopath/src/github.com/chaincode/
    ports:
        - 8888:12345
    networks:
        - basic

Last point worth to mention is the ``setup.sh`` part where ``cryptogen`` and ``configtxgen`` are used. The first tool, based on the documentation: "cryptogen is an utility for generating Hyperledger Fabric key material. It is provided as a means of preconfiguring a network for testing purposes. It would normally not be used in the operation of a production network.". The second tool, based on the documentation: "The configtxgen command allows users to create and inspect channel config related artifacts. The content of the generated artifacts is dictated by the contents of configtx.yaml.". In our example, we generate the genesis block and the channel creation configuration.

``cryptogen generate --config=./crypto-config.yaml``

``configtxgen -profile OneOrgOrdererGenesis -outputBlock ./config/genesis.block``

``configtxgen -profile OneOrgChannel -outputCreateChannelTx ./config/channel.tx -channelID channel1``

``configtxgen -profile OneOrgChannel -outputAnchorPeersUpdate ./config/Org1MSPanchors.tx -channelID channel1 -asOrg Org1MSP``